In [33]:
import openai

In [34]:
client = openai.Client()

Vector_store foi movido de beta para namespace principal

In [35]:
vector_store =  client.vector_stores.create(
    name = "Desinger UX"
)

In [36]:
file = ["files/heuristicas.pdf"]
file_stream = [open(f,'rb') for f in file]
file_batch = client.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id,
    files=file_stream
)

In [37]:
file_batch.status

'completed'

In [38]:
file_batch.file_counts

FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)

In [39]:
assistant = client.beta.assistants.create(
    name = "Desinger UX",
    instructions = "Voce é um experiente desinger User experience.",
    tools = [{"type":"file_search"}],
    tool_resources={"file_search":{"vector_store_ids":[vector_store.id]}},
    model="gpt-4-turbo-preview"
 )

In [59]:
# pergunta = "Com base no documento, quais as principais heuristicas?"
pergunta = "Qual tem mais impacto para o usuário?"

In [60]:
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content = pergunta
)

C:\Users\frpbo\AppData\Local\Temp\ipykernel_21912\1131807865.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
C:\Users\frpbo\AppData\Local\Temp\ipykernel_21912\1131807865.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(


In [61]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id, 
    instructions="Premium"
)

C:\Users\frpbo\AppData\Local\Temp\ipykernel_21912\1460489382.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


In [62]:
import time

while run.status in ["queued", "in_progress", "cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id, 
        run_id=run.id)
    

C:\Users\frpbo\AppData\Local\Temp\ipykernel_21912\641873556.py:5: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(


In [64]:
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id = thread.id
    )
    print(mensagens)
else:
    print(f'Erro {run.status}')

C:\Users\frpbo\AppData\Local\Temp\ipykernel_21912\3548958572.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  mensagens = client.beta.threads.messages.list(


SyncCursorPage[Message](data=[Message(id='msg_n9S4vskcEEiPMcbGtsh94u7i', assistant_id='asst_ey0WXWbzvUrTZ4F1R9R9Bnbm', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=632, file_citation=FileCitation(file_id='file-VFBpLgUjyGbSqP8X2ctdDy'), start_index=611, text='【4:2†heuristicas.pdf】', type='file_citation'), FileCitationAnnotation(end_index=939, file_citation=FileCitation(file_id='file-VFBpLgUjyGbSqP8X2ctdDy'), start_index=918, text='【4:3†heuristicas.pdf】', type='file_citation'), FileCitationAnnotation(end_index=1172, file_citation=FileCitation(file_id='file-VFBpLgUjyGbSqP8X2ctdDy'), start_index=1151, text='【4:0†heuristicas.pdf】', type='file_citation')], value='As 10 heurísticas de Nielsen cobrem uma ampla gama de aspectos importantes para melhorar a usabilidade de interfaces digitais, o que tem um grande impacto na experiência do usuário. Cada heurística aborda diferentes elementos cruciais para a criação de um produt

In [65]:
print(mensagens.data[0].content[0].text.value)

As 10 heurísticas de Nielsen cobrem uma ampla gama de aspectos importantes para melhorar a usabilidade de interfaces digitais, o que tem um grande impacto na experiência do usuário. Cada heurística aborda diferentes elementos cruciais para a criação de um produto digital eficaz e agradável. Aqui estão alguns destaques que exemplificam o impacto desses princípios:

1. **Visibilidade do status do sistema**: Manter os usuários informados sobre o que está acontecendo, oferecendo feedbacks instantâneos. Isso orienta o usuário sobre a ação que está acontecendo e é essencial para manter a confiança na interface【4:2†heuristicas.pdf】.

2. **Estética e design minimalista**: Um bom design não é apenas sobre aparência; também é sobre funcionalidade. Um site da Apple, por exemplo, é frequentemente citado por seu design minimalista que não apenas parece bom, mas também facilita a localização de informações pelo usuário【4:3†heuristicas.pdf】.

3. **Ajuda os usuários a reconhecer, diagnosticar e recupe

In [66]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

C:\Users\frpbo\AppData\Local\Temp\ipykernel_21912\2960082042.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_steps = client.beta.threads.runs.steps.list(


In [75]:
for step in run_steps.data:
    print(f"\n📌 Step: {step.step_details.type}")

    if step.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )

        for content in message.content:
            if content.type == "text":
                print("💬 Resposta:")
                print(content.text.value)

            elif content.type == "file_citation":
                print("📄 Citação de arquivo:")
                print(f"🔹 Trecho: {content.file_citation.quote}")
                print(f"🔹 Arquivo ID: {content.file_citation.file_id}")

    elif step.type == "tool_calls":
        for call in step.step_details.tool_calls:
            print("AQUI",call.type)
            print(call)
            if call.type == "code_interpreter":
                print("🧮 Código executado:")
                print(call.code_interpreter.input)
            elif call.type == "file_search":
                for file in call.file_search.results:
                    print(f'{file.file_id} - {file.score}' )
                    print("📂 Busca vetorial foi executada com sucesso.")



📌 Step: message_creation


C:\Users\frpbo\AppData\Local\Temp\ipykernel_21912\3652955666.py:5: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.retrieve(


💬 Resposta:
As 10 heurísticas de Nielsen cobrem uma ampla gama de aspectos importantes para melhorar a usabilidade de interfaces digitais, o que tem um grande impacto na experiência do usuário. Cada heurística aborda diferentes elementos cruciais para a criação de um produto digital eficaz e agradável. Aqui estão alguns destaques que exemplificam o impacto desses princípios:

1. **Visibilidade do status do sistema**: Manter os usuários informados sobre o que está acontecendo, oferecendo feedbacks instantâneos. Isso orienta o usuário sobre a ação que está acontecendo e é essencial para manter a confiança na interface【4:2†heuristicas.pdf】.

2. **Estética e design minimalista**: Um bom design não é apenas sobre aparência; também é sobre funcionalidade. Um site da Apple, por exemplo, é frequentemente citado por seu design minimalista que não apenas parece bom, mas também facilita a localização de informações pelo usuário【4:3†heuristicas.pdf】.

3. **Ajuda os usuários a reconhecer, diagnosti